In [1]:
! pip install dalex pymysql --user >> /dev/null

In [2]:
import pandas as pd
import time
import random
import os, requests, json
import datetime
import joblib
from sqlalchemy import create_engine

In [3]:
test_df = pd.read_csv("test.csv",names=['GeneID', 'Bin ID', 'H3K27me3 count', 'H3K36me3 count', 'H3K4me1 count', 'H3K4me3 count', 'H3K9me3 counts', 'Binary Label'])
input_features = ['GeneID', 'Bin ID', 'H3K27me3 count', 'H3K36me3 count', 'H3K4me1 count', 'H3K4me3 count', 'H3K9me3 counts']

In [4]:
test_df.head()

,GeneID,Bin ID,H3K27me3 count,H3K36me3 count,H3K4me1 count,H3K4me3 count,H3K9me3 counts,Binary Label
0,172936,1,0,6,1,2,0,0
1,172936,2,0,10,1,0,2,0
2,172936,3,0,12,1,0,3,0
3,172936,4,0,13,0,2,2,0
4,172936,5,0,4,0,4,4,0


In [5]:
hostname = "10.142.0.3"
databasename = "monitoring2"
username = "root"
password = "test"
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                            .format(host=hostname, db=databasename, user=username, pw=password))
try:
    if engine.connect():
        print("Connection to SQL is successful")
except Exception as e:
    print(e)

Connection to SQL is successful


In [6]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
predict_url = "https://10.142.0.3:32222/dkube/inference/abhijeetc/feb271b6fd:predict"

In [7]:
no_of_samples = random.randint(10,15)
print(no_of_samples)

11


In [8]:
sample_df = test_df.sample(no_of_samples)
sample_df

,GeneID,Bin ID,H3K27me3 count,H3K36me3 count,H3K4me1 count,H3K4me3 count,H3K9me3 counts,Binary Label
843,172969,44,5,5,2,2,9,0
222,172939,23,0,0,0,0,0,0
80,172936,81,1,10,0,4,4,0
335,172940,36,6,2,11,2,0,0
473,172943,74,0,0,0,0,0,1
197,172938,98,3,0,0,2,1,0
883,172969,84,27,1,2,14,10,0
126,172938,27,8,0,1,2,3,0
545,172954,46,7,0,8,28,1,1
696,172955,97,0,0,0,0,0,0


In [9]:
values = sample_df[input_features].values
print(values)

[[172969     44      5      5      2      2      9]
 [172939     23      0      0      0      0      0]
 [172936     81      1     10      0      4      4]
 [172940     36      6      2     11      2      0]
 [172943     74      0      0      0      0      0]
 [172938     98      3      0      0      2      1]
 [172969     84     27      1      2     14     10]
 [172938     27      8      0      1      2      3]
 [172954     46      7      0      8     28      1]
 [172955     97      0      0      0      0      0]
 [172943     65      2      0      0      2      1]]


In [10]:
while True:
    no_of_samples = random.randint(10,15)
    sample_df = test_df.sample(no_of_samples)
    values = sample_df[input_features].values
    payload = {"instances": values.tolist(), "token": token}
    r = requests.post(predict_url, json=payload, 
                      headers = {'authorization': "Bearer " + token}, 
                      verify = False)
    if r.status_code == 200:
        start = datetime.datetime.utcnow()
        end = start + datetime.timedelta(seconds=10)
        predictions = r.json()["predictions"]
        labels = sample_df["Binary Label %"]
        timestamps = pd.date_range(start, end, len(predictions))
        labelled_df = pd.DataFrame({
            "DateTime": timestamps,
            "Binary Label %": predictions,
            "Binary Label": labels
        })
        labelled_df.to_sql("attentive", engine, if_exists="append", index=False)
        print("labelled data pushed")
    time.sleep(60)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.142.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.142.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.142.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/usr/local/lib/python3.6/dist-packages/urllib3/connectionp

KeyboardInterrupt: 

In [6]:
import  kipoi
model = kipoi.get_model("AttentiveChrome/E005")

Cloning into '/home/abhijeetc/.kipoi/models'...
Checking out files: 100% (443/443), done.


238kB [00:02, 107kB/s]                             
/usr/local/lib/python3.8/site-packages/torch/nn/modules/rnn.py:57: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [7]:
dl_dictionary = model.default_dataloader.example_kwargs #This is an example dataloader.
print("dl_dictionary:", dl_dictionary)

prediction = model.pipeline.predict(dl_dictionary)

24.6kB [00:01, 14.6kB/s]                            


dl_dictionary: {'input_file': '/home/abhijeetc/.kipoi/models/AttentiveChrome/downloaded/example_files/input_file'}


0it [00:00, ?it/s]

Number of genes: 10
Number of entries: 1000
Number of HMs: 7


1it [00:00,  3.98it/s]


In [8]:
prediction

array([[0.6430358 ],
       [0.04643877],
       [0.67054904],
       [0.2707719 ],
       [0.6830173 ],
       [0.55665994],
       [0.18436134],
       [0.13774122],
       [0.06227126],
       [0.60603684]], dtype=float32)

In [ ]:
while True:
   
    
#     payload = {"instances": values.tolist(), "token": token}
#     r = requests.post(predict_url, json=payload, 
#                       headers = {'authorization': "Bearer " + token}, 
#                       verify = False)
    sc=200
    dl_dictionary = model.default_dataloader.example_kwargs #This is an example dataloader.
    print("dl_dictionary:", dl_dictionary)
    model.pipeline.predict({'input_file': '/test.csv','bin_size':100})
    prediction = model.pipeline.predict(dl_dictionary)
    if sc == 200:
        start = datetime.datetime.utcnow()
        end = start + datetime.timedelta(seconds=10)
#         predictions = r.json()["predictions"]
        labels = sample_df["No-show %"]
        timestamps = pd.date_range(start, end, len(predictions))
        labelled_df = pd.DataFrame({
            "AppointmentDateTime": timestamps,
            "No-show %": predictions,
            "No-show-label": labels
        })
        labelled_df.to_sql("no_show_label", engine, if_exists="append", index=False)
        print("labelled data pushed")
    time.sleep(60)